In [1]:
import astropy
import datetime
from stokespy import instload
import sunpy
import time

from astropy.io import fits
from astropy.time import Time, TimeDelta
from scipy import ndimage
from sunpy.image import coalignment
import matplotlib.pyplot as plt
import numpy as np
import drms, glob, os

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_image_shift(image1, image2):
    # slightly modified code snippet from bitbucket to determine the shift
    # between two images using FFT:
    
    # discrete fast fourier transformation and complex conjugation of image2 
    image1FFT = np.fft.fft2(image1)
    image2FFT = np.conjugate( np.fft.fft2(image2) )

    # inverse fourier transformation of product -> equal to cross correlation
    imageCCor = np.real( np.fft.ifft2( (image1FFT*image2FFT) ) )

    # shift the zero-frequency component to the center of the spectrum
    imageCCorShift = np.fft.fftshift(imageCCor)

    # determine the distance of the maximum from the center
    row, col = image1.shape

    yshift, xshift = np.unravel_index( np.argmax(imageCCorShift), (row,col) )

    xshift -= int(col/2)
    yshift -= int(row/2)

    return xshift, yshift

# Load the hinode files

In [2]:
# Choose the Hinode file to load.
user_dir = os.getcwd() + '/Data/Hinode/'
#user_date = '20170822_171104'   # AR near limb
user_date = '20170905_030404'  # AR near center

### Generate the scan file list.
level1_dir = user_dir + 'Level1/' + user_date
level1_files = []
for file in sorted(os.listdir(level1_dir)):
    if not file.endswith(".fits"):
        continue
    level1_files.append(os.path.join(level1_dir, file))

In [35]:
# Choose the Hinode file to load.

### Read the Level 2 fit data. 
level2_fname = user_dir + '/Level2/' + user_date + '.fits'
SP_level2 = astropy.io.fits.open(level2_fname)

Ny, Nx = SP_level2['Field_Strength'].data.shape
level2_data = np.zeros((4, Ny, Nx))
level2_data[0] = SP_level2['Field_Strength'].data
level2_data[1] = SP_level2['Field_Inclination'].data
level2_data[2] = SP_level2['Field_Azimuth'].data
level2_data[3] = SP_level2['Stray_Light_Fill_Factor'].data
level2_data.shape

### Build WCS objects.
#head1 = SP_level1['PRIMARY'].header
#head2 = SP_level2['PRIMARY'].header

(4, 512, 485)

In [ ]:
magnetic_flux_density = alpha*B*np.cos(np.radians(theta))

In [34]:
# Stray_Light_Fill_Factor
SP_level2[12].header

XTENSION= 'IMAGE   '           / IMAGE extension                                
BITPIX  =                  -32 /  IEEE single precision floating point          
NAXIS   =                    2 / Number of data axes                            
NAXIS1  =                  485 / Number of positions along axis 1               
NAXIS2  =                  512 / Number of positions along axis 2               
PCOUNT  =                    0 / No Group Parameters                            
GCOUNT  =                    1 / One Data Group                                 
EXTNAME = 'Stray_Light_Fill_Factor' / Merlin : Stray Light Fill Factor          

In [9]:
np.mean(level2_data[2,:,:] - alpha)

86.06435608435794

In [36]:
# Load the Hinode HDU
hdulist_hinode = fits.open(level2_fname)
B = hdulist_hinode[1].data
theta = hdulist_hinode[2].data
alpha = hdulist_hinode[12].data  # Filling factor

magnetic_flux_density = alpha*B*np.cos(np.radians(theta))
print(magnetic_flux_density.shape)

In [37]:
magnetic_flux_density

array([[ -1.249706  ,  -1.4289496 ,  -1.5557722 , ...,   1.0675883 ,
          2.0682337 ,   1.8580909 ],
       [ -1.249706  ,  -1.4289496 ,  -1.5557722 , ...,   1.0675883 ,
          2.0682337 ,   1.8580909 ],
       [ -1.249706  ,  -1.4289496 ,  -1.5557722 , ...,   1.0675883 ,
          2.0682337 ,   1.8580909 ],
       ...,
       [ -7.0834355 ,   2.3293    ,  -8.924343  , ...,   0.16180603,
         -7.0464897 ,  -7.229126  ],
       [ -5.1937466 ,   9.411543  , -11.369891  , ..., -60.989174  ,
        -18.107742  ,  -5.2353964 ],
       [  2.8998919 ,   5.850771  ,   8.145     , ..., -38.627598  ,
         -9.80321   ,  -3.88478   ]], dtype=float32)

# Load the HMI file

In [7]:
# Select a date and time to search for observations.
#user_date = astropy.time.Time(datetime.datetime(2016, 7, 28, 23, 57, 0), scale='tai')  # Original
#user_date = astropy.time.Time(datetime.datetime(2017, 8, 22, 17, 11, 4), scale='tai')  # AR near limb
user_date = astropy.time.Time(datetime.datetime(2017, 9, 5, 3, 4, 4), scale='tai')  # AR near disc center

# Find the nearest set of Stokes and corresponding inversion results.
#all_fnames_stokes, all_fnames_magvec = get_HMI_data(user_date, user_notify='gdima@hawaii.edu', download=False)
#lv1_data, lv1_wcs, lv2_data, lv2_wcs = \
#    get_HMI_data(user_date, user_notify='gdima@hawaii.edu', download=False, show_files=False)

time0 = astropy.time.Time(user_date.gps - 1., format='gps', scale='tai')
time1 = astropy.time.Time(user_date.gps + 1., format='gps', scale='tai')

user_dir = os.getcwd() + '/Data/SDO/'

all_fnames_stokes = instload.parse_folder(dir_path=user_dir, inst='hmi', \
                                 series='S_720s', ext='fits', show=False)

if len(all_fnames_stokes) > 1:
    tstamps = [i.split('.')[2] for i in all_fnames_stokes]
    tstamps = [sunpy.time.parse_time('_'.join(i.split('_')[0:2])) for i in tstamps]
    tstamps_diff = [np.abs(i.gps - user_date.gps) for i in tstamps]

# Search for the closest timestamp
tstamps_diff = np.asarray(tstamps_diff)
tstamps_ix, = np.where(tstamps_diff == tstamps_diff.min())

all_fnames_stokes = np.asarray(all_fnames_stokes)[tstamps_ix]

derotate = True

In [13]:
# Load Level1 files.

# Check for individual timestamps.
unique_tstamps = []
for i in range(len(all_fnames_stokes)):
    if all_fnames_stokes[i].split('.')[2] not in unique_tstamps:
        unique_tstamps.append(all_fnames_stokes[i].split('.')[2])

print(f'No download requested. Found {len(all_fnames_stokes)} Stokes files with unique timestamp(s):')
print(unique_tstamps)

tstart = time.time()
level1_data = []
for i, fname in enumerate(all_fnames_stokes):
    tmp_map = sunpy.map.Map(fname)
    if derotate:
        tmp_map = tmp_map.rotate(order=3)
    level1_data.append(tmp_map.data)

level1_data = np.asarray(level1_data)
level1_data = level1_data.reshape(4,6,level1_data.shape[1], level1_data.shape[2])

print(f'Created Stokes data cube with dimensions: {level1_data.shape}')

print(time.time() - tstart)

No download requested. Found 24 Stokes files with unique timestamp(s):
['20170905_030000_TAI']


Created Stokes data cube with dimensions: (4, 6, 4098, 4098)
36.09508728981018


In [16]:
## Create the WCS object
# Expand the coordinate axis to include wavelength and stokes dimensions.

l0 = 6173.345 * 1.e-10  # m Central wavelength for FeI line
dl = 0.0688   * 1.e-10  # m 

# Generate WCS for data cube using same WCS celestial information from AIA map.
# This reads the header from the last tmp_map created (and maybe rotated) above.
wcs_header = tmp_map.wcs.to_header()

wcs_header["WCSAXES"] = 4

# Add wavelength axis.
wcs_header["CRPIX3"] = 3.5
wcs_header["CDELT3"] = dl
wcs_header["CUNIT3"] = 'm'
wcs_header["CTYPE3"] = "WAVE"
wcs_header["CRVAL3"] = l0

# Add Stokes axis.
wcs_header["CRPIX4"] = 0
wcs_header["CDELT4"] = 1
wcs_header["CUNIT4"] = ''
wcs_header["CTYPE4"] = "STOKES"
wcs_header["CRVAL4"] = 0

level1_wcs = astropy.wcs.WCS(wcs_header)

In [17]:
level1_wcs

WCS Keywords

Number of WCS axes: 4
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  'WAVE'  'STOKES'  
CRVAL : 0.0  0.0  6.173345e-07  0.0  
CRPIX : 2057.7563593149  2047.3749883579  3.5  0.0  
PC1_1 PC1_2 PC1_3 PC1_4  : 1.0  7.4040177438477e-22  0.0  0.0  
PC2_1 PC2_2 PC2_3 PC2_4  : 7.4040177438477e-22  1.0  0.0  0.0  
PC3_1 PC3_2 PC3_3 PC3_4  : 0.0  0.0  1.0  0.0  
PC4_1 PC4_2 PC4_3 PC4_4  : 0.0  0.0  0.0  1.0  
CDELT : 0.00014009532001283  0.00014009532001283  6.88e-12  1.0  
NAXIS : 0  0

In [8]:
# Load Level2 files.

all_fnames_magvec = instload.parse_folder(dir_path=user_dir, inst='hmi', 
                                          show=False, series='ME_720s_fd10', ext='fits')
        
if len(all_fnames_magvec) > 1:
    tstamps = [i.split('.')[2] for i in all_fnames_magvec]
    tstamps = [sunpy.time.parse_time('_'.join(i.split('_')[0:2])) for i in tstamps]
    tstamps_diff = [np.abs(i.gps - user_date.gps) for i in tstamps]
else:
    print('No files found close to the date requested')

# Search for the closest timestamp
tstamps_diff = np.asarray(tstamps_diff)
tstamps_ix, = np.where(tstamps_diff == tstamps_diff.min())

all_fnames_magvec = np.asarray(all_fnames_magvec)[tstamps_ix]

unique_tstamps = []
for i in range(len(all_fnames_stokes)):
    if all_fnames_magvec[i].split('.')[2] not in unique_tstamps:
        unique_tstamps.append(all_fnames_magvec[i].split('.')[2])
print(f'No download requested. Found {len(all_fnames_magvec)} inversion files with timestamps: ')
print(unique_tstamps)


# Create MagVectorCube from HMI inversions
#mag_params = ['field', 'inclination', 'azimuth']
mag_params = ['field', 'inclination', 'azimuth']

level2_data = []

# Load 2D maps into level2_data in the order determined by entries in mag_params
if derotate:
    print('OBS: Derotating each magnetic field image.')
use_fnames = []
for mag_param in mag_params:
    for i, fname in enumerate(all_fnames_magvec):
        data_id = fname.split('.')[-2]
        if data_id == mag_param:
            use_fnames.append(fname)
            tmp_map = sunpy.map.Map(fname)
            if derotate:
                tmp_map = tmp_map.rotate(order=3)
            level2_data.append(tmp_map.data)

level2_data = np.asarray(level2_data)

print(f'Created data cube with dimensions: {level2_data.shape}')
#print('Filenames used: ')

No download requested. Found 25 inversion files with timestamps: 
['20170905_030000_TAI']
OBS: Derotating each magnetic field image.
Created data cube with dimensions: (3, 4098, 4098)


In [16]:
tmp_map = sunpy.map.Map(use_fnames[0])

In [12]:
level2_data.shape

(3, 4098, 4098)

In [19]:
tmp_map.wcs

WCS Keywords

Number of WCS axes: 2
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  
CRVAL : 0.0  0.0  
CRPIX : 2040.244140625  2050.626953125  
PC1_1 PC1_2  : -0.99999997235056  0.00023515712208659  
PC2_1 PC2_2  : -0.00023515712208659  -0.99999997235056  
CDELT : 0.00014009532001283  0.00014009532001283  
NAXIS : 0  0

In [56]:
use_fnames[1]

'/home/gabriel/Desktop/Science/StokesPY/stokespy_notebooks/Data/SDO/hmi.me_720s_fd10.20170905_030000_TAI.inclination.fits'

In [9]:
# Load the HMI HDU

hmi_inclination_hdu = fits.open(use_fnames[1])
hmi_field_hdu = fits.open(use_fnames[0])

hmi_inclination = hmi_inclination_hdu[1].data
hmi_field = hmi_field_hdu[1].data

# Now compute the flux density for HMI.
# Projected along LOS
magnetic_flux_density_hmi = hmi_field*np.cos(np.radians(hmi_inclination))

In [10]:
hmi_field_hdu[1].header['CROTA2']

180.0134735107422

In [11]:
magnetic_flux_density_hmi.shape

(4096, 4096)

In [12]:
# Rotate the HMI image based on the rotate keyword in the header.
hmi_rot_angle = hmi_field_hdu[1].header['CROTA2']
magnetic_flux_density_hmi_rot = ndimage.rotate(np.nan_to_num(magnetic_flux_density_hmi), \
                                                hmi_rot_angle, reshape=False)

In [17]:
magnetic_flux_density_hmi_rot.shape

(4096, 4096)

In [18]:
%matplotlib widget

#plt.figure()
plt.imshow(magnetic_flux_density_hmi_rot, cmap="gray", vmin=-10., vmax=10., origin='lower') 
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Calculate bounding coordinates

In [19]:
# Hinode coordinates.

hinode_xcoords = [np.quantile(hdulist_hinode[38].data,0.01)-1., np.quantile(hdulist_hinode[38].data,0.99)+1.]
hinode_ycoords = [np.quantile(hdulist_hinode[39].data,0.01)-1., np.quantile(hdulist_hinode[39].data,0.99)+1.]
print(hinode_xcoords)
print(hinode_ycoords)

[55.729801177978516, 204.052001953125]
[-40.919643325805666, 121.69612915039062]


In [20]:
# HMI coordinates.
# Great, HMI is rotated by ~180 degrees, but only sometimes ¯\_(ツ)_/¯
# Introduce a very simple switch for the calculation of the coordinates
# here because I am too lazy to do this properly right now.

#hmi_rot_angle = hmi_field_hdu[1].header['CROTA2']
crval1 = hmi_field_hdu[1].header['CRVAL1']
crpix1 = hmi_field_hdu[1].header['CRPIX1']
cdelt1 = hmi_field_hdu[1].header['CDELT1']
crval2 = hmi_field_hdu[1].header['CRVAL2']
crpix2 = hmi_field_hdu[1].header['CRPIX2']
cdelt2 = hmi_field_hdu[1].header['CDELT2']


if hmi_rot_angle > 170.:
    tmp_hmi_xcoords = crval1 + (crpix1 - np.arange(hmi_field.shape[0], dtype=float))*cdelt1
    tmp_hmi_ycoords = crval2 + (crpix2 - np.arange(hmi_field.shape[1], dtype=float))*cdelt2

    hmi_xcoords = tmp_hmi_xcoords*np.cos(np.radians(hmi_rot_angle)) - \
      tmp_hmi_ycoords*np.sin(np.radians(hmi_rot_angle)) 
    hmi_ycoords = tmp_hmi_xcoords*np.sin(np.radians(hmi_rot_angle)) + \
      tmp_hmi_ycoords*np.cos(np.radians(hmi_rot_angle)) 
else:
    hmi_xcoords = crval1 + (np.arange(hmi_field.shape[0], dtype=float) - crpix1)*cdelt1
    hmi_ycoords = crval2 + (np.arange(hmi_field.shape[1], dtype=float) - crpix2)*cdelt2

In [21]:
# Find out which indices in HMI the Hinode coordinates correspond to.
hmi_index_x = [np.argmin(np.abs(np.array(hmi_xcoords)-np.floor(hinode_xcoords[0]))), \
                np.argmin(np.abs(np.array(hmi_xcoords)-np.ceil(hinode_xcoords[1])))]

hmi_index_y = [np.argmin(np.abs(np.array(hmi_ycoords)-np.floor(hinode_ycoords[0]))), \
                np.argmin(np.abs(np.array(hmi_ycoords)-np.ceil(hinode_ycoords[1])))]

In [22]:
hmi_index_x, hmi_index_y

([2149, 2447], [1969, 2292])

In [23]:
# Degrade the Hinode data with a Gaussian and rescale to the same pixel size as HMI.
# The 1.3 sigma for the Gaussian is not actually chosen based on any physical parameters,
# just such that the Hinode map looks ~ish like HMI.
tmp_hinode_data_0 = ndimage.gaussian_filter(magnetic_flux_density,1.3)
tmp_hinode_data = ndimage.zoom(tmp_hinode_data_0, \
                                [(hmi_index_y[1]-hmi_index_y[0])/tmp_hinode_data_0.shape[0], \
                                    (hmi_index_x[1]-hmi_index_x[0])/tmp_hinode_data_0.shape[1]])

print('Zoom values: ', [(hmi_index_y[1]-hmi_index_y[0])/tmp_hinode_data_0.shape[0], \
                                    (hmi_index_x[1]-hmi_index_x[0])/tmp_hinode_data_0.shape[1]])

Zoom values:  [0.630859375, 0.6144329896907217]


In [101]:
magnetic_flux_density.shape

(512, 485)

In [24]:
tmp_hinode_data.shape

(323, 298)

In [25]:
hmi_index_y[1]-hmi_index_y[0], hmi_index_x[1]-hmi_index_x[0]

(323, 298)

In [26]:
tmp_hinode_data_0.shape

(512, 485)

In [27]:
# Now add 150 pixels padding on either side to allow for the Hinode offset wrt HMI. 
# Technically this could hit the HMI array boundaries and there is no failsafe 
# implemented, but HMI maps are so large that for the HOP maps we will probably
# not hit the sides. At least it does not for the 153 HOP north maps...
pix_pad = 150 
hmi_data_template = magnetic_flux_density_hmi_rot[hmi_index_y[0]-pix_pad:hmi_index_y[1]+pix_pad, \
                                                    hmi_index_x[0]-pix_pad:hmi_index_x[1]+pix_pad]


In [28]:
hmi_data_template.shape

(623, 598)

In [29]:
%matplotlib widget
plt.imshow(hmi_data_template, origin='lower')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
# Use the sunpy coalignment routine to get an initial idea where the degraded Hinode data
# should be on the padded HMI map.  
thisxyshift = coalignment.calculate_shift(hmi_data_template, tmp_hinode_data)

# Use that information to create a hopefully more precise HMI cutout.
hmi_index_x_new = np.around(hmi_index_x+(thisxyshift[0].value-pix_pad)).astype(int)
hmi_index_y_new = np.around(hmi_index_y+(thisxyshift[1].value-pix_pad)).astype(int)
hmi_data = magnetic_flux_density_hmi_rot[hmi_index_y_new[0]:hmi_index_y_new[1],hmi_index_x_new[0]:hmi_index_x_new[1]]


In [31]:
hmi_index_x, thisxyshift[0].value-pix_pad

([2149, 2447], 55.437541160256785)

In [32]:
hmi_index_y, thisxyshift[1].value-pix_pad

([1969, 2292], 50.658641096472735)

In [119]:
thisxyshift

(<Quantity 205.43754116 pix>, <Quantity 200.6586411 pix>)

In [121]:
hmi_data.shape, tmp_hinode_data.shape

((323, 298), (323, 298))

# Plot the results

In [33]:
# the comparison plot
hinode_data = magnetic_flux_density

fig, ax = plt.subplots(1, 3, constrained_layout=True, figsize=(9,3.5))
ax[0].imshow(hinode_data, cmap="gray", origin='lower')
ax[1].imshow(tmp_hinode_data, cmap="gray", origin='lower')
ax[2].imshow(hmi_data, cmap="gray", origin='lower')
ax[0].title.set_text('Hinode (full res)')
ax[1].title.set_text('Hinode (degraded)')
ax[2].title.set_text('HMI')
for ii in range(0,3):
    ax[ii].get_xaxis().set_visible(False)
    ax[ii].get_yaxis().set_visible(False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …